## Classification Part 2

1. Here I am going to try and build on my clustering attempts but using actual classification models.
1. Updated data from Data Cleaning to use same data as Jesse
1. Try to beat the prediction accuracy of Clustering. 
1. Find the best basic model.


## Further Expantion in Part 3
1. Move from a basic model to the more advanced models.
1. Split data into time of year or try to get seasonal trends out of the data that can skew things.
1. Finalize the model and accuracy.

## Thoughts at this point. End of Update 1

1. First off this was always going to be hard because income level does not determine what you do for fun in your free time. Some people have a lot of money and watch a lot of TV. Some people have no money or Job so they watch a lot of TV. 
1. I think there are quite a few outliers so I think maybe we do something to separate those out of the dataset and try again.
1. Split Into averages for time of day I think this might get us closer to what we are looking for. 
1. Make sure each cluster has close to the same number of values? 
    1. So far there has always been 1 with far few values than the others. 
    1. I think that this coinsides with the outliers forcing a centroid to far away from the rest of the data.

In [68]:
import pandas as pd
import plotly.express as px
import numpy as np

# Find the correct sklearn packages.
# KNN, Decision Tree, RandomForest, SVM
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
data_frame = pd.read_csv("final_data.csv")

In [3]:
data_frame.head()

,DateTime,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped,Day,Month
0,2013-01-01 00:00:00,MAC000769,Std,0.014,ACORN-F,Comfortable,Tuesday,January
1,2013-01-01 00:30:00,MAC000769,Std,0.000,ACORN-F,Comfortable,Tuesday,January
2,2013-01-01 01:00:00,MAC000769,Std,0.044,ACORN-F,Comfortable,Tuesday,January
3,2013-01-01 01:30:00,MAC000769,Std,0.036,ACORN-F,Comfortable,Tuesday,January
4,2013-01-01 02:00:00,MAC000769,Std,0.029,ACORN-F,Comfortable,Tuesday,January


In [22]:
data_frame["groups"] = np.where(data_frame["Acorn_grouped"] == "Affluent", 1, np.where(data_frame["Acorn_grouped"] == "Comfortable", 2, 3))

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [23]:
X = data_frame.groupby("LCLid").agg({"KWH/hh": ["mean", "max"], "groups": "first"})

In [27]:
X.head()

KWH/hh        groups
               mean    max  first
LCLid                            
MAC000002  0.240788  2.994      1
MAC000006  0.060175  0.774      3
MAC000010  0.532447  4.189      2
MAC000018  0.212908  2.088      1
MAC000019  0.131976  1.641      3

In [34]:
X = X.droplevel(0, axis=1)

In [67]:
X.head()

,mean,max,first
LCLid,,,
MAC000002,0.240788,2.994,1
MAC000006,0.060175,0.774,3
MAC000010,0.532447,4.189,2
MAC000018,0.212908,2.088,1
MAC000019,0.131976,1.641,3


In [37]:
x = X[["mean", "max"]]

In [69]:
Xtrain, Xtest, y_train, y_test = train_test_split(X[["mean", "max"]], X["first"], random_state = 5, train_size = 0.8)

In [70]:
model = KNeighborsClassifier(n_neighbors=2)
model.fit(Xtrain, y_train)


KNeighborsClassifier(n_neighbors=2)

In [71]:
knn_pred = model.predict(Xtest)


In [72]:
accuracy_score(y_test, knn_pred)

0.40625

40% testing accuracy with base settings

In [73]:
from sklearn import tree

In [139]:
dt_model = tree.DecisionTreeClassifier(min_samples_split=5, random_state=7)

In [140]:
dt_model.fit(Xtrain, y_train)

DecisionTreeClassifier(min_samples_split=5, random_state=7)

In [141]:
dt_preds = dt_model.predict(Xtest)

In [142]:
accuracy_score(y_test, dt_preds)

0.32589285714285715

32% testing accuracy with base settings

In [143]:
dt_model.get_depth()

21

In [144]:
from sklearn.ensemble import RandomForestClassifier

In [145]:
rf_model = RandomForestClassifier(n_estimators=100, min_samples_split=5,max_depth=10, random_state=0)

In [146]:
rf_model.fit(Xtrain, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=0)

In [147]:
rf_preds = dt_model.predict(Xtest)

In [148]:
accuracy_score(y_test, rf_preds)

0.32589285714285715

In [156]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [157]:
svm_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [158]:
svm_model.fit(Xtrain, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [159]:
svm_preds = svm_model.predict(Xtest)

In [160]:
accuracy_score(y_test, svm_preds)

0.4375

In [161]:
rf_preds

array([3, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 2, 2, 3, 2, 3, 3, 3, 1, 2, 2, 2,
       1, 1, 1, 3, 3, 1, 3, 3, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 1, 2,
       3, 3, 2, 3, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 2, 3, 2, 2, 3, 1, 3, 1,
       1, 3, 2, 2, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 3, 1, 1, 2, 1,
       3, 2, 1, 3, 2, 1, 1, 3, 1, 1, 2, 1, 3, 2, 1, 2, 3, 3, 2, 3, 2, 2,
       2, 3, 1, 2, 1, 1, 3, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1,
       3, 3, 3, 1, 1, 2, 1, 3, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 3, 2, 3,
       3, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 1, 3, 3, 3,
       1, 1, 1, 1, 1, 2, 3, 2, 1, 3, 1, 1, 2, 2, 2, 1, 2, 3, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 2, 2, 1, 2, 2, 2, 1, 2, 3,
       2, 2, 3, 2])

In [162]:
y_test.to_numpy()

array([1, 1, 1, 3, 3, 2, 2, 3, 3, 3, 2, 3, 1, 2, 3, 3, 2, 3, 1, 3, 1, 2,
       1, 3, 3, 3, 3, 2, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 2, 2, 3, 1, 3, 1, 3, 1, 2, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 3, 1, 2, 2, 3, 3, 2, 3, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2,
       1, 1, 1, 3, 1, 2, 3, 2, 3, 1, 3, 1, 1, 2, 3, 2, 1, 1, 1, 3, 3, 2,
       2, 2, 1, 2, 2, 2, 1, 1, 3, 1, 1, 3, 3, 1, 1, 2, 3, 1, 3, 1, 3, 2,
       3, 2, 3, 3, 3, 1, 1, 2, 2, 3, 2, 2, 3, 2, 1, 1, 1, 3, 3, 3, 3, 3,
       2, 1, 3, 1, 1, 1, 2, 1, 3, 3, 2, 3, 2, 3, 1, 1, 3, 2, 1, 1, 2, 1,
       3, 1, 2, 1, 3, 2, 3, 3, 1, 3, 1, 1, 1, 1, 3, 2, 1, 1, 2, 1, 3, 3,
       2, 2, 1, 3, 3, 1, 2, 3, 1, 2, 2, 3, 2, 3, 3, 1, 1, 3, 1, 3, 3, 2,
       1, 1, 1, 1])

In [163]:
dt_preds

array([3, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 2, 2, 3, 2, 3, 3, 3, 1, 2, 2, 2,
       1, 1, 1, 3, 3, 1, 3, 3, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 1, 2,
       3, 3, 2, 3, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 2, 3, 2, 2, 3, 1, 3, 1,
       1, 3, 2, 2, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 3, 1, 1, 2, 1,
       3, 2, 1, 3, 2, 1, 1, 3, 1, 1, 2, 1, 3, 2, 1, 2, 3, 3, 2, 3, 2, 2,
       2, 3, 1, 2, 1, 1, 3, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1,
       3, 3, 3, 1, 1, 2, 1, 3, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 3, 2, 3,
       3, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 1, 3, 3, 3,
       1, 1, 1, 1, 1, 2, 3, 2, 1, 3, 1, 1, 2, 2, 2, 1, 2, 3, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 2, 2, 1, 2, 2, 2, 1, 2, 3,
       2, 2, 3, 2])

In [164]:
knn_pred

array([1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 2, 2, 3, 3, 2, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1,
       2, 1, 2, 2, 3, 3, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       1, 2, 1, 2, 1, 2, 2, 1, 3, 2, 3, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1,
       3, 1, 3, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 3, 2,
       2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 1,
       2, 2, 1, 3, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 3, 3, 1, 2, 1, 1, 3, 1, 3,
       1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 3,
       1, 1, 1, 2])

In [165]:
svm_preds

array([3, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 3, 3, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       3, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3,
       1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3, 3,
       3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 3, 1, 3, 1, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 1, 3, 3, 1, 1, 3, 1, 2, 3, 3, 1, 1, 1, 3, 1, 3, 3, 1, 1, 3,
       1, 1, 3, 1])

In [169]:
X["first"].value_counts() / X["first"].count()

1    0.406082
3    0.313059
2    0.280859
Name: first, dtype: float64

So for Update 2 I updated the data to use the same data as Jesse. Then I went through 4 base models trying to find the best one for predicting which class a household belongs to based on mean and max KWH/hh. The assumption is that we should be able to randomly pick one of the 3 bins and have an accuracy of 33%. So as long as we are beating that then the model seems worth using in some capacity. Based on that the KNN and SVM models we both better than random chance at 40 and 43% accuracy respectively. DT and RF were both at 32% which is basically the same as random chance so they are not currently good models at all. 

I think the next steps are to see if one model is better than the others at predicting a class. Then we can actually build a model where each of the above models vote to make a prediction with maybe some bias to SVM or something along those lines. In theory the worst case should be SVM of 43% but we might be able to better that percentage using the mdoels in unison. 

We also might want to consider seasonal effects in the time series that might shine some extra light on these models. It is possible that certain groups are more consistent throughout the year or time of day and that can teach us something or make us better at predicting. 

In [172]:
y_train.value_counts() / y_train.count()

1    0.410515
3    0.305369
2    0.284116
Name: first, dtype: float64

In [173]:
y_test.value_counts() / y_test.count()

1    0.388393
3    0.343750
2    0.267857
Name: first, dtype: float64

In [175]:
Xtest

,mean,max
LCLid,,
MAC004730,0.076228,1.186
MAC002563,0.237328,2.296
MAC003603,0.771411,4.037
MAC002576,0.184566,1.845
MAC001773,0.095766,0.934
...,...,...
MAC002388,0.006913,0.011
MAC001665,0.624861,3.532
MAC004102,0.578222,4.021


In [176]:
Xtest["target"] = y_test

In [185]:
Xtest.head()

,mean,max,target,knn,dt,rf,svm,knn_correct,dt_correct,rf_correct,svm_correct
LCLid,,,,,,,,,,,
MAC004730,0.076228,1.186,1,1,3,3,3,True,False,False,False
MAC002563,0.237328,2.296,1,1,1,1,1,True,True,True,True
MAC003603,0.771411,4.037,1,1,1,1,1,True,True,True,True
MAC002576,0.184566,1.845,3,1,1,1,1,False,False,False,False
MAC001773,0.095766,0.934,3,1,2,2,3,False,False,False,True


In [181]:
Xtest["knn"] = knn_pred
Xtest["dt"] = dt_preds
Xtest["rf"] = rf_preds
Xtest["svm"] = svm_preds

In [184]:
Xtest["knn_correct"] = Xtest.knn == Xtest.target
Xtest["dt_correct"] = Xtest.dt == Xtest.target
Xtest["rf_correct"] = Xtest.rf == Xtest.target
Xtest["svm_correct"] = Xtest.svm == Xtest.target

In [197]:
px.scatter(Xtest, x="max", y="mean", color="knn_correct", opacity=0.6, symbol='target', category_orders={"knn_correct": [True, False]})

In [198]:
px.scatter(Xtest, x="max", y="mean", color="svm_correct", opacity=0.6, symbol='target', category_orders={"svm_correct": [True, False]})

In [199]:
px.scatter(Xtest, x="max", y="mean", color="dt_correct", opacity=0.6, symbol='target',category_orders={"dt_correct": [True, False]})